In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
import pandas as pd
from copy import deepcopy

In [2]:
def euclidean_distance(X, Y):
    return cdist(X, Y, 'euclidean')

In [3]:
def cosine_similarity_distance(X, Y):
    return 1 - cdist(X, Y, 'cosine')

In [4]:
def jaccard_similarity(X, Y):
    min_int = np.minimum(X[:, np.newaxis], Y)
    max_union = np.maximum(X[:, np.newaxis], Y)
    return 1 - np.sum(min_int, axis=2) / np.sum(max_union, axis=2)

In [5]:
def kmeans(X, k, distance_function, max_iters=100):
    n_samples, _ = X.shape

    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]

    centroids_old = np.zeros(centroids.shape)

    labels = np.zeros(n_samples)
    error = np.linalg.norm(centroids - centroids_old)

    iteration = 0
    while error != 0 and iteration < max_iters:
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)
        centroids_old = deepcopy(centroids)
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)

        error = np.linalg.norm(centroids - centroids_old)
        iteration += 1

    return centroids, labels

In [6]:
def calculate_sse(X, labels, centroids, distance_function):
    sse = 0
    for i in range(len(centroids)):
        cluster_points = X[labels == i]
        distances = distance_function(cluster_points, [centroids[i]])
        sse += np.sum(distances**2)
    return sse

In [8]:
data_path = 'kmeans_data/data.csv'
label_path = 'kmeans_data/label.csv'

data_df = pd.read_csv(data_path)
label_df = pd.read_csv(label_path)

data = data_df.to_numpy()
labels = label_df.to_numpy().flatten() 

In [9]:
X = data_df.to_numpy()
k = 10


centroids_euclidean, labels_euclidean = kmeans(data, k, euclidean_distance) # Euclidian Distance
centroids_cosine, labels_cosine = kmeans(data, k, cosine_similarity_distance) # Cosine Similarity
centroids_jaccard, labels_jaccard = kmeans(data, k, jaccard_similarity) # Jaccard Similarity

print(labels_euclidean[:10])  
print(labels_cosine[:10])    
print(labels_jaccard[:10])   

/Users/shreyassrinivasan/Desktop/course/dm/hw3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/shreyassrinivasan/Desktop/course/dm/hw3/lib/python3.9/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/bc/hpyl601n6w30mtc7cx5q8l7w0000gn/T/ipykernel_25278/2312079844.py:42: RuntimeWarning: invalid value encountered in cast
  centroids[i] = np.mean(X[labels == i], axis=0)


[2 6 5 9 6 7 7 9 0 5]
[0 0 0 0 0 0 0 0 0 0]
[9 1 8 7 3 0 0 5 4 8]


Q1


In [10]:

sse_euclidean = calculate_sse(data, labels_euclidean, centroids_euclidean, euclidean_distance)
sse_cosine = calculate_sse(data, labels_cosine, centroids_cosine, cosine_similarity_distance)
sse_jaccard = calculate_sse(data, labels_jaccard, centroids_jaccard, generalized_jaccard_similarity)

print("SSE Euclidean is:", sse_euclidean)
print("SSE Cosine is:", sse_cosine)
print("SSE Jaccard is:", sse_jaccard)

SSE Euclidean is: 25486250479.0
SSE Cosine is: 4156.594182164835
SSE Jaccard is: 3687.566716929624


Q2

In [11]:
def label_clusters(labels, true_labels, k):
    cluster_labels = {}
    for i in range(k):
        labels_in_cluster = true_labels[labels == i]
        if len(labels_in_cluster) == 0:
            continue
        most_common = np.bincount(labels_in_cluster).argmax()
        cluster_labels[i] = most_common
    return cluster_labels


In [12]:
def compute_accuracy(labels, true_labels, cluster_labels):
    correct_predictions = 0
    for label, true_label in zip(labels, true_labels):
        if cluster_labels[label] == true_label:
            correct_predictions += 1
    return correct_predictions / len(true_labels)


In [13]:
true_labels_numeric = pd.factorize(labels)[0]
cluster_labels_euclidean = label_clusters(labels_euclidean, true_labels_numeric, k)
cluster_labels_cosine = label_clusters(labels_cosine, true_labels_numeric, k)
cluster_labels_jaccard = label_clusters(labels_jaccard, true_labels_numeric, k)

accuracy_euclidean = compute_accuracy(labels_euclidean, true_labels_numeric, cluster_labels_euclidean)
accuracy_cosine = compute_accuracy(labels_cosine, true_labels_numeric, cluster_labels_cosine)
accuracy_jaccard = compute_accuracy(labels_jaccard, true_labels_numeric, cluster_labels_jaccard)

print("Accuracy Euclidean:", accuracy_euclidean)
print("Accuracy Cosine:", accuracy_cosine)
print("Accuracy Jaccard:", accuracy_jaccard)

Accuracy Euclidean: 0.6034603460346034
Accuracy Cosine: 0.11351135113511351
Accuracy Jaccard: 0.5446544654465446


Q3


In [14]:
import time

def kmeans_time(X, k, distance_function, max_iters=500):
    n_samples, _ = X.shape
    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]
    centroids_old = np.zeros(centroids.shape)
    labels = np.zeros(n_samples)
    error = np.linalg.norm(centroids - centroids_old)
    iteration = 0
    sse_old = float('inf')

    start_time = time.time()

    while iteration < max_iters:
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)
        centroids_old = deepcopy(centroids)
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)
        error = np.linalg.norm(centroids - centroids_old)
        sse_new = calculate_sse(X, labels, centroids, distance_function)

        if error == 0 or sse_new > sse_old:
            break

        sse_old = sse_new
        iteration += 1

    end_time = time.time()
    return centroids, labels, iteration, end_time - start_time

In [15]:
centroids_euclidean, labels_euclidean, iterations_euclidean, time_euclidean = kmeans_time(data, k, euclidean_distance)
centroids_cosine, labels_cosine, iterations_cosine, time_cosine = kmeans_time(data, k, cosine_similarity_distance)
centroids_jaccard, labels_jaccard, iterations_jaccard, time_jaccard = kmeans_time(data, k, jaccard_similarity)

print(f"Euclidean: Iterations are - {iterations_euclidean}, Time taken to converge - {time_euclidean} seconds")
print(f"Cosine: Iterations are - {iterations_cosine}, Time taken to converge - {time_cosine} seconds")
print(f"Jaccard: Iterations are - {iterations_jaccard}, Time taken to converge - {time_jaccard} seconds")

/var/folders/bc/hpyl601n6w30mtc7cx5q8l7w0000gn/T/ipykernel_25278/1556918395.py:20: RuntimeWarning: invalid value encountered in cast
  centroids[i] = np.mean(X[labels == i], axis=0)


Euclidean: Iterations are - 105, Time taken to converge - 30.738922834396362 seconds
Cosine: Iterations are - 1, Time taken to converge - 0.9287569522857666 seconds
Jaccard: Iterations are - 12, Time taken to converge - 22.971154928207397 seconds


Q4


In [16]:
def kmeans_cond(X, k, distance_function, max_iters=100):
    n_samples, _ = X.shape
    random_indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[random_indices]
    centroids_old = np.zeros(centroids.shape)
    labels = np.zeros(n_samples)
    error = np.linalg.norm(centroids - centroids_old)
    iteration = 0
    sse_old = float('inf')
    sse_at_termination = None

    while iteration < max_iters:
        distances = distance_function(X, centroids)
        labels = np.argmin(distances, axis=1)
        centroids_old = deepcopy(centroids)
        for i in range(k):
            centroids[i] = np.mean(X[labels == i], axis=0)
        error = np.linalg.norm(centroids - centroids_old)
        sse_new = calculate_sse(X, labels, centroids, distance_function)

        if error == 0 or sse_new > sse_old:
            sse_at_termination = sse_new
            break

        sse_old = sse_new
        iteration += 1

        if iteration == max_iters - 1:
            sse_at_termination = sse_new

    return centroids, labels, sse_at_termination


In [17]:
centroids_euclidean, labels_euclidean, sse_euclidean = kmeans_cond(data, k, euclidean_distance)
centroids_cosine, labels_cosine, sse_cosine = kmeans_cond(data, k, cosine_similarity_distance)
centroids_jaccard, labels_jaccard, sse_jaccard = kmeans_cond(data, k, generalized_jaccard_similarity)

print(f"SSE Euclidean is: {sse_euclidean}")
print(f"SSE Cosine is: {sse_cosine}")
print(f"SSE Jaccard is: {sse_jaccard}")


SSE Euclidean is: 25433664655.0
SSE Cosine is: 5058.586215879547
SSE Jaccard is: 3665.779439501196
